In [ ]:
!pip install duckdb

## Setup

### Dataset

In [1]:
import torch
import duckdb
import pandas as pd


In [2]:
con = duckdb.connect('choon_dataset.db')

con.sql('SHOW TABLES')

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ choon_images     │
│ not_choon_images │
└──────────────────┘

In [3]:
choon_df = (
    con.sql('FROM choon_images').df()
    .assign(label=1)
)
not_choon_df = (
    con.sql('FROM not_choon_images').df()
    .assign(label=0)
)

all_df = (
    pd.concat([choon_df, not_choon_df])
    .reset_index(drop=True)
)

In [4]:
all_df

,image_tensors,label
0,"[0.7764706, 0.78039217, 0.78039217, 0.78039217...",1
1,"[0.7764706, 0.78039217, 0.76862746, 0.7647059,...",1
2,"[0.73333335, 0.74509805, 0.7529412, 0.7372549,...",1
3,"[0.83137256, 0.8392157, 0.84313726, 0.8352941,...",1
4,"[0.85490197, 0.8509804, 0.8509804, 0.8509804, ...",1
...,...,...
384,"[0.7019608, 0.69411767, 0.69803923, 0.69803923...",0
385,"[0.11764706, 0.11764706, 0.14509805, 0.1960784...",0
386,"[0.4862745, 0.49019608, 0.49019608, 0.49019608...",0
387,"[0.38039216, 0.3764706, 0.3529412, 0.3647059, ...",0


In [31]:
all_shuffled = all_df.sample(frac=1, random_state=3).reset_index(drop=True)

train_frac = 0.7  # 70% for training
test_frac = 0.15  # 15% for testing
validate_frac = 0.15  # 15% for validation

total_len = len(df_shuffled)
train_len = int(total_len * train_frac)
test_len = int(total_len * test_frac)
validate_len = total_len - train_len - test_len

train_df = all_shuffled[:train_len]
test_df = all_shuffled[train_len:train_len+test_len]
valid_df = all_shuffled[train_len+test_len:]

dfs = {
    'train': train_set,
    'valid': validate_set,
    'test': test_set
}

for name, df in dfs.items():
    percentages = df.label.value_counts() / len(df)
    percentages = round(percentages, 2)
    percent_0, percent_1 = percentages[0], percentages[1]
    print(f"{name}")
    print(f'\tlen =   {len(df)}')
    print(f"\t% 0's = {percent_0}\n\t% 1's = {percent_1}")

train
	len =   272
	% 0's = 0.54
	% 1's = 0.46
valid
	len =   59
	% 0's = 0.47
	% 1's = 0.53
test
	len =   58
	% 0's = 0.57
	% 1's = 0.43
